# Quick start

*EPAM Syngen* is an unsupervised tabular data generation tool based on a variational autoencoder (VAE). 
It supports common tabular datatypes (floats, ints, datetime, text, categorical, binary) and can generate linked tables that sharing keys using the simple statistical approach. 
The SDK exposes simple programmatic entry points for training, inference, report generation, loading and saving data in supported formats - *CSV*, *Avro* and *Excel* format. The data should be located locally and be in UTF-8 encoding.

This notebook demonstrates the SDK usage. Install the package and then you can call the main SDK class `Syngen` to run training, inference or generation of reports, and the class`DataIO` to load and save the data in supported formats.

Python *3.10* or *3.11* is required to run the library. The library is tested on Linux and Windows operating systems.

# Installation

Please, install the library *syngen* (from Pypi):

In [13]:
!pip install  --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ --use-pep517 --no-cache-dir syngen==0.10.28rc0

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/


## Launch training

You can start training process using the SDK entrypoint `Syngen.train(...)`. This will train a model and save the model artifacts to a disk. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
Syngen.train(
    metadata_path: Optional[str] = None,  # use a metadata file in the '.yaml', '.yml' format to train multiple tables or provide all params in a file
    table_name: Optional[str] = None,     # required for single-table training
    source: Optional[str] = None,         # path to original data
    epochs: int = 10,                     # number of training epochs
    drop_null: bool = False,              # whether to drop rows with at least one missing value
    row_limit: Optional[int] = None,      # a number of rows to train over
    reports: Union[str, List[str]] = "none",  # report types: 'none', 'accuracy', 'sample', 'metrics_only', 'all'
    log_level: str = "INFO",              # logging level
    batch_size: int = 32,                 # training batch size
    fernet_key: Optional[str] = None      # name of the environment variable containing the Fernet key for secure storage of the data subset
)
```

### Parameters description:

- **`metadata_path`** *(Optional[str], default: None)*: Path to a metadata file in the *'.yaml'* or *'.yml'* format for training multiple tables or specifying per-table settings. If provided, individual table parameters (`source`, `table_name`) are ignored in favor of metadata values.

- **`table_name`** *(Optional[str], default: None)*: **Required for single-table training**. An arbitrary name used to identify the table and name the artifact folders. Must be provided when not using `metadata_path`.

- **`source`** *(Optional[str], default: None)*: **Required for single-table training**. Path to the original data file.

- **`epochs`** *(int, default: 10)*: Number of training epochs. Must be ≥ 1. Since the early stopping mechanism is implemented the bigger value of epochs is the better.

- **`drop_null`** *(bool, default: False)*: Whether to drop rows containing at least one missing value before training. When `False`, missing values are handled during the training process.

- **`row_limit`** *(Optional[int], default: None)*: Maximum number of rows to use for training. If specified and less than the total rows, a random subset of the specified size will be selected. Useful for testing or working with large datasets.

- **`reports`** *(Union[str, List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing synthetic data (same size as original) with original dataset to estimate the quality of training process
  - `"sample"` - generates a sample report showing distribution comparisons between the original data and the subset of this data
  - `"metrics_only"` - outputs metrics to stdout
  - `"all"` - generates both accuracy and sample reports
  - List example: `["accuracy", "sample"]` to generate multiple report types
  - *Note*: Report generation may require significant time for large tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: Logging level for the training process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`batch_size`** *(int, default: 32)*: Training batch size. Must be ≥ 1. Splits training into batches to optimize memory usage. Smaller batches use less RAM but may increase training time.

- **`fernet_key`** *(Optional[str], default: None)*: Name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is encrypted on disk (stored in the `.dat` format). If not provided, data is stored unencrypted as `.pkl` format. **Important**: The same key must be used during inference and report generation to decrypt the data.

### Key notes:

- **Single table training**: Both `source` and `table_name` are required when `metadata_path` is not provided.
- **Multiple tables training**: Use `metadata_path` to define multiple tables with or without relationships.

*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [14]:
# The example of training the single table

from syngen.sdk import Syngen

Syngen.train(
    source="../examples/example-data/housing.csv", 
    table_name="housing", 
    epochs=5,
    drop_null=False,
    row_limit=1000, 
    batch_size=32, 
    reports="all"
)

2025-10-15 14:14:01.505 | INFO     | syngen.train:launch_train:69 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-15 14:14:01.512 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/resources/housing/' was removed
2025-10-15 14:14:01.515 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2025-10-15 14:14:01.516 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2025-10-15 14:14:01.516 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not 

32/32 [==============================] - 0s 712us/step


2025-10-15 14:14:14.905 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2025-10-15 14:14:14.905 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2025-10-15 14:14:16.509 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2025-10-15 14:14:16.556 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing/vae/checkpoints
2025-10-15 14:14:16.556 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2025-10-15 14:14:16.557 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2025-10-15 14:14:16.557 | INFO     | syngen.ml.strategies.strategies:run:160 - Training of the table - 'housing' was completed
2025-10-15 14:14:16.558 | INFO     | syngen.ml.worker.worker:_infer_table:518 - Infer process of the table - 'housing' has started
2025-10-15 14:14:

32/32 [==============================] - 0s 1ms/step


2025-10-15 14:14:16.966 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2025-10-15 14:14:16.966 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2025-10-15 14:14:16.966 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 1
2025-10-15 14:14:16.966 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:14:16.967 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 8353.67it/s]
2025-10-15 14:14:17.193 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:14:17.198 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing' was completed. Synthetic data s

In [1]:
# The example of training of multiple tables with relationships

from syngen.sdk import Syngen

Syngen.train(
    metadata_path="../examples/example-metadata/housing_metadata.yaml",
    log_level="DEBUG"
)

2025-10-15 14:23:51.083060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-15 14:23:51.083096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-15 14:23:51.083961: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-15 14:23:53.366 | INFO     | syngen.train:launch_train:69 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
20

25/25 [==============================] - 0s 839us/step


2025-10-15 14:24:05.753 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2025-10-15 14:24:05.753 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2025-10-15 14:24:06.482 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2025-10-15 14:24:06.523 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-properties/vae/checkpoints
2025-10-15 14:24:06.523 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2025-10-15 14:24:06.523 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2025-10-15 14:24:06.523 | INFO     | syngen.ml.strategies.strategies:run:160 - Training of the table - 'housing_properties' was completed
2025-10-15 14:24:06.533 | INFO     | syngen.ml.processors.processors:_preprocess_data:120 - As the parameter 'drop_null' set t

 1/57 [..............................] - ETA: 3s


2025-10-15 14:24:18.604 | INFO     | syngen.ml.vae.wrappers.wrappers:_train:410 - epoch: 4, total loss: 3.4235310554504395, time: 2.3696 sec
2025-10-15 14:24:18.606 | INFO     | syngen.ml.vae.models.model:fit_sampler:179 - Fit sampler
2025-10-15 14:24:18.610 | INFO     | syngen.ml.vae.models.model:fit_sampler:183 - Start encoding


...

2025-10-15 14:24:20.402 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2025-10-15 14:24:20.402 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=790, run_parallel=False, batch_size=790, random_seed=1
2025-10-15 14:24:20.402 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2025-10-15 14:24:20.403 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 1
2025-10-15 14:24:20.403 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:24:20.403 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 9033.89it/s]
2025-10-15 14:24:20.567 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table 

32/32 [==============================] - 0s 879us/step


Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4991.24it/s]

25/25 [==============================] - 0s 923us/step



2025-10-15 14:24:20.883 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 2 of 2
2025-10-15 14:24:20.883 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:24:20.883 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4822.89it/s]
2025-10-15 14:24:20.958 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2025-10-15 14:24:20.976 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing_conditions' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv'
2025-10-15 14:24:20.977 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-15 14:2

# Launch inference

You can start inference process using the SDK entrypoint `Syngen.infer(...)`. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
Syngen.infer(
    metadata_path: Optional[str] = None,  # use a metadata file in the '.yaml', '.yml' to infer multiple tables or provide all params in a file
    table_name: Optional[str] = None,     # required for single-table inference
    size: int = 100,                      # the desired number of rows to generate
    run_parallel: bool = False,           # whether to use multiprocessing (feasible for tables > 5000 rows)
    batch_size: Optional[int] = None,     # inference batch size
    random_seed: Optional[int] = None,    # if specified, generates a reproducible result
    reports: Union[str, List[str]] = "none",  # report types: 'none', 'accuracy', 'metrics_only', 'all'
    log_level: str = "INFO",              # logging level
    fernet_key: Optional[str] = None      # name of the environment variable containing the Fernet key for decrypting the data subset
)
```

### Parameters description:

- **`metadata_path`** *(Optional[str], default: None)*: Path to a metadata file in the *'.yaml'* or *'.yml'* format for inferring multiple tables or specifying per-table settings. If provided, the individual table parameter `table_name` is ignored in favor of metadata values.

- **`table_name`** *(Optional[str], default: None)*: **Required for single-table inference**. The name of the table to generate synthetic data for. Must match the name used during training.

- **`size`** *(int, default: 100)*: The desired number of synthetic rows to generate. Must be ≥ 1.

- **`run_parallel`** *(bool, default: False)*: Whether to use multiprocessing for data generation. Set to `True` to enable parallel processing, which is recommended and feasible for generating large tables (>5000 rows).

- **`batch_size`** *(Optional[int], default: None)*: Inference batch size. Must be ≥ 1. If specified, the generation is split into batches to optimize memory usage and save RAM.

- **`random_seed`** *(Optional[int], default: None)*: Random seed for reproducible generation. Must be ≥ 0.

- **`reports`** *(Union[str, List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing original and synthetic data patterns to verify quality of a generated data
  - `"metrics_only"` - outputs metrics to stdout without generating an accuracy report
  - `"all"` - generates an accuracy report (same as `"accuracy"`)
  - List example: `["accuracy", "metrics_only"]` to generate multiple report types
  - *Note*: Report generation may require significant time for large generated tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: Logging level for the inference process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`fernet_key`** *(Optional[str], default: None)*: Name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is decrypted for report generation. **Important**: The same key used during training must be used during inference to successfully decrypt the data.

### Key notes:

- **Single table inference**: `table_name` is required when `metadata_path` is not provided, and it must match the name used during the training process.
- **Multiple tables inference**: Use `metadata_path` to generate multiple tables with or without relationships simultaneously.

*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [ ]:
# The example of inference for the single table
from syngen.sdk import Syngen

Syngen.infer(
    table_name="housing", 
    size=200,
    run_parallel=False,
    batch_size=32, 
    random_seed=42,
    reports="all"
)

2025-10-15 14:13:23.537 | INFO     | syngen.infer:launch_infer:43 - The inference process will be executed according to the information mentioned in 'infer_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-15 14:13:23.539 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/merged_infer_housing.csv' was removed
2025-10-15 14:13:23.540 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/infer_message.success' was removed
2025-10-15 14:13:23.541 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:172 - As the destination path wasn't specified for the table - 'housing', the synthetic data will be stored at the default path - './model_a

1/1 [==============================] - 0s 124ms/step


2025-10-15 14:13:23.970 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2025-10-15 14:13:23.971 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 7 batch(es)
2025-10-15 14:13:23.971 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 7
2025-10-15 14:13:23.971 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:13:23.972 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13133.32it/s]

1/1 [==============================] - 0s 13ms/step



2025-10-15 14:13:24.125 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.125 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 2 of 7
2025-10-15 14:13:24.126 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:13:24.126 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 14458.58it/s]
2025-10-15 14:13:24.166 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.166 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 3 of 7
2025-10-15 14:13:24.167 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2

...


2025-10-15 14:13:24.249 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.250 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 5 of 7
2025-10-15 14:13:24.250 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:13:24.250 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 17549.39it/s]
2025-10-15 14:13:24.289 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.290 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 6 of 7
2025-10-15 14:13:24.290 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2

1/1 [==============================] - 0s 13ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 15328.02it/s]
2025-10-15 14:13:24.328 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.328 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 7 of 7
2025-10-15 14:13:24.328 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:13:24.328 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.

2025-10-15 14:13:24.328 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.328 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 7 of 7
2025-10-15 14:13:24.328 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2

1/1 [==============================] - 0s 125ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 14936.01it/s]
2025-10-15 14:13:24.481 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.484 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2025-10-15 14:13:24.484 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-15 14:13:24.484 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing' has started

2025-10-15 14:13:24.481 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:13:24.484 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesi

In [2]:
# The example of inference of multiple tables with relationships

from syngen.sdk import Syngen

Syngen.infer(
    metadata_path="../examples/example-metadata/housing_metadata.yaml",
    log_level="DEBUG"
)

2025-10-15 14:24:56.206 | INFO     | syngen.infer:launch_infer:43 - The inference process will be executed according to the information mentioned in 'infer_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-15 14:24:56.213 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:300 - The schema of the metadata is valid
2025-10-15 14:24:56.214 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/merged_infer_housing-properties.csv' was removed
2025-10-15 14:24:56.214 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/infer_message.success' was removed
2025-10-15 14:24:56.214 | INFO     | syngen.ml.worker.worker:_r

1/1 [==============================] - 0s 95ms/step


2025-10-15 14:24:56.792 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2025-10-15 14:24:56.792 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=90, run_parallel=False, batch_size=32, random_seed=10, reports - 'accuracy'
2025-10-15 14:24:56.793 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 3 batch(es)
2025-10-15 14:24:56.793 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 3
2025-10-15 14:24:56.793 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:24:56.793 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 10404.01it/s]

1/1 [==============================] - 0s 14ms/step



2025-10-15 14:24:56.917 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 2 of 3
2025-10-15 14:24:56.917 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:24:56.917 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 13437.13it/s]
2025-10-15 14:24:56.957 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 3 of 3
2025-10-15 14:24:56.958 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:24:56.958 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.

2025-10-15 14:24:56.957 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the 

...

Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 7472.04it/s]

1/1 [==============================] - 0s 13ms/step



2025-10-15 14:24:57.361 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 2 of 3
2025-10-15 14:24:57.361 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:24:57.361 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 8377.44it/s]
2025-10-15 14:24:57.399 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 3 of 3
2025-10-15 14:24:57.400 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:24:57.400 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.

2025-10-15 14:24:57.399 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the b

1/1 [==============================] - 0s 62ms/step


Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 9124.66it/s]
2025-10-15 14:24:57.492 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2025-10-15 14:24:57.496 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing_conditions' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv'
2025-10-15 14:24:57.497 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-15 14:24:57.497 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing_conditions' has started

2025-10-15 14:24:57.492 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2025-10-15 14:24:57.496 | INFO     | syngen.ml.strategies.strategies:run:242 - Synth

# Data security: Using Fernet Key for encryption

In the current implementation, a sample of the original data is stored on a disk during the training process. To ensure data security and protect sensitive information, you can use a **Fernet key** to encrypt this data.

## What is a Fernet Key?

A Fernet key is a 44-character URL-safe base64-encoded string used for symmetric encryption. When provided, the data subset is encrypted on a disk (stored in `.dat` format instead of unencrypted `.pkl` format).

## How to Generate a Fernet Key

You can generate a Fernet key using the following code:

In [7]:
# Generate a Fernet key
from cryptography.fernet import Fernet

fernet_key = Fernet.generate_key().decode("utf-8")

## Setting the Fernet Key as an environment variable

After generating the key, you need to store it as an environment variable. This can be done in your terminal or programmatically in Python.

### Option 1: Set in Terminal (Linux/macOS)

```bash
export MY_FERNET_KEY='your_generated_fernet_key_here'
```

### Option 2: Set in Terminal (Windows)

```cmd
set MY_FERNET_KEY=your_generated_fernet_key_here
```

### Option 3: Set programmatically in Python

```python
import os
os.environ['MY_FERNET_KEY'] = 'your_generated_fernet_key_here'
```

## Using the Fernet Key in a training

When training with encryption, pass the name of the environment variable (not the key itself) to the `fernet_key` parameter:

In [5]:
# The example: the training with the Fernet key encryption

import os
from syngen.sdk import Syngen

# Step 1: Set the Fernet key as an environment variable
os.environ['MY_FERNET_KEY'] = fernet_key  # Using the key generated above

# Step 2: Train with encryption enabled
Syngen.train(
    source="../examples/example-data/housing.csv", 
    table_name="housing_encrypted", 
    epochs=5,
    row_limit=1000, 
    batch_size=32, 
    fernet_key="MY_FERNET_KEY"  # Pass the environment variable name, not the key itself
)

2025-10-15 14:31:06.505 | INFO     | syngen.train:launch_train:69 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-15 14:31:06.509 | WARNING  | syngen.ml.config.validation:_launch_validation:394 - Encryption and decryption are enabled for the table 'housing_encrypted' as a Fernet key is provided
2025-10-15 14:31:06.509 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2025-10-15 14:31:06.510 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2025-10-15 14:31:06.510 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the track

32/32 [==============================] - 0s 795us/step


2025-10-15 14:31:19.891 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2025-10-15 14:31:19.891 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2025-10-15 14:31:21.746 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2025-10-15 14:31:21.786 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-encrypted/vae/checkpoints
2025-10-15 14:31:21.786 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2025-10-15 14:31:21.786 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2025-10-15 14:31:21.786 | INFO     | syngen.ml.strategies.strategies:run:160 - Training of the table - 'housing_encrypted' was completed
2025-10-15 14:31:21.789 | INFO     | syngen.ml.data_loaders.data_loaders:save_data:681 - Data is successfully encrypted and save

## Using the Fernet Key in an inference

**Important**: When generating synthetic data with inference, you must use the **same Fernet key** that was used during training. This allows the system to decrypt the stored data subset for report generation.

If the Fernet key is not provided or doesn't match the training key, the inference process will fail when trying to access the encrypted data.

In [6]:
# Example: Inference with Fernet key decryption

from syngen.sdk import Syngen

# The environment variable 'MY_FERNET_KEY' is already set from the training step

# Inference with the same Fernet key
Syngen.infer(
    table_name="housing_encrypted",  # Must match the name used in training
    size=200,
    batch_size=32, 
    random_seed=42,
    reports="all",
    fernet_key="MY_FERNET_KEY"  # Must use the same key as in training
)

2025-10-15 14:33:48.162 | INFO     | syngen.infer:launch_infer:43 - The inference process will be executed according to the information mentioned in 'infer_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-15 14:33:48.165 | WARNING  | syngen.ml.config.validation:_launch_validation:394 - Encryption and decryption are enabled for the table 'housing_encrypted' as a Fernet key is provided
2025-10-15 14:33:48.166 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:172 - As the destination path wasn't specified for the table - 'housing_encrypted', the synthetic data will be stored at the default path - './model_artifacts/tmp_store/housing-encrypted/merged_infer_housing-encrypted.csv'
2025-10-15 14:33:48.167 | INFO     | syngen.ml.data_loaders.data_loaders:load_data:706 - Data stored at the path - 'model_arti

1/1 [==============================] - 0s 131ms/step


2025-10-15 14:33:48.624 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-encrypted/vae/checkpoints
2025-10-15 14:33:48.625 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 7 batch(es)
2025-10-15 14:33:48.625 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 1 of 7
2025-10-15 14:33:48.625 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:33:48.626 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 12692.53it/s]
2025-10-15 14:33:48.789 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:33:48.789 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'hous

1/1 [==============================] - 0s 14ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13642.03it/s]

...


2025-10-15 14:33:48.923 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:33:48.924 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 5 of 7
2025-10-15 14:33:48.924 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:33:48.924 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 11557.45it/s]
2025-10-15 14:33:48.969 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:33:48.969 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 6 of 7
2025-10-15 14:33:48.970 | INFO     | syngen.ml.handlers.handlers:ru

1/1 [==============================] - 0s 14ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13330.64it/s]
2025-10-15 14:33:49.013 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:33:49.014 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 7 of 7
2025-10-15 14:33:49.014 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-15 14:33:49.014 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.


1/1 [==============================] - 0s 138ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13530.01it/s]
2025-10-15 14:33:49.182 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-15 14:33:49.185 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing_encrypted' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-encrypted/merged_infer_housing-encrypted.csv'
2025-10-15 14:33:49.186 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-15 14:33:49.186 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing_encrypted' has started
2025-10-15 14:33:49.187 | INFO     | syngen.ml.data_loaders.data_loaders:load_data:706 - Data stored at the path - 'model_artifacts/tmp_store/housing-encrypted/input_data_housing-encrypted.dat' has been 

## Using the Fernet Key with the metadata file

You can also specify the Fernet key in the metadata file for both training and inference:

```yaml
global:
  encryption:
    fernet_key: MY_FERNET_KEY  # Name of the environment variable

TABLE_NAME:
  train_settings:
    source: "./data/table.csv"
  
  infer_settings:
    size: 100
  
  # You can also specify per-table encryption
  encryption:
    fernet_key: MY_FERNET_KEY
```

Then use it in your code:

```python
# Training with the metadata file and the Fernet key
Syngen.train(metadata_path="path/to/metadata.yaml")

# Inference with the metadata file and the Fernet key
Syngen.infer(metadata_path="path/to/metadata.yaml")
```

## Important Security Notes

⚠️ **Critical Security Considerations:**

1. **Store the key securely**: Never hardcode the Fernet key directly in your code or commit it to version control systems.
2. **Key recovery is impossible**: If you lose the Fernet key, encrypted data cannot be recovered
3. **Same key required**: Always use the same Fernet key for training and inference
4. **Environment variables**: Use environment variables to store the key securely
5. **Key length**: The Fernet key must be exactly 44 characters (URL-safe base64-encoded)
6. **Production environments**: In production, use secure secret management services (AWS Secrets Manager, Azure Key Vault, HashiCorp Vault, etc.)

## What Happens Without a Fernet Key?

If you don't provide a `fernet_key` parameter:
- Data subset is stored **unencrypted** in `.pkl` format
- No decryption is needed during inference
- Suitable for non-sensitive data or development environments

With a `fernet_key`:
- Data subset is stored **encrypted** in `.dat` format
- Decryption is required during inference using the same key
- Recommended for sensitive or production data